# 🚀 GENERATE ASSETS (OG)
-------

In [1]:
# import nltk

# # Download required NLTK data (only need to do this once)
# nltk.download('punkt')

In [2]:
import trendspy
from trendspy import Trends
import os
import datetime
from dotenv import load_dotenv
from news_utils import News
import feedparser
import time
import pickle
from generateAssets import *
load_dotenv()

c:\Users\garre\OneDrive\Desktop\Projects\aiMedia


True

In [3]:
# Make directory for the run
APPDIR = os.getenv("APPDIR")
BASEPATH = os.getenv("BASEPATH")
RUN_DIR = os.path.join(BASEPATH, datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
os.makedirs(BASEPATH, exist_ok=True)
os.makedirs(RUN_DIR, exist_ok=True)
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
VOICE_ID = os.getenv("ELEVENLABS_VOICE_ID")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.chdir(APPDIR)

In [4]:
os.environ["RUN_DIR"] = RUN_DIR

In [5]:
def getTrends():
    def freeze(d : dict):
        return 
    tr = Trends()
    trends = tr.trending_now(geo='US', hours=24)
    businessFiltered = trends.filter_by_topic('Business and Finance')
    techFiltered = trends.filter_by_topic('Technology')
    lawAndGovFiltered = trends.filter_by_topic('Law and Government')
    politicsFiltered = trends.filter_by_topic('Politics')
    filteredTrends = businessFiltered + techFiltered + lawAndGovFiltered + politicsFiltered
    sortedTrends = sorted(filteredTrends, key=lambda x: x.volume, reverse=True)
    keysSoFar = set()
    finalTrendList = []
    for trend in sortedTrends:
        if not trend.keyword in keysSoFar:
            finalTrendList.append({
                'keyword': trend.keyword,
                'volume': trend.volume,
                'volume_growth_pct': trend.volume_growth_pct,
                'topics': trend.topics,
                'keywords': trend.trend_keywords,
            })
            keysSoFar.add(trend.keyword)
    return finalTrendList

In [6]:
# Example usage in your getNews function:
def getNews(trendKeyword, num_articles=10, maxAge=10, linkNews={}):
    #maxAge is the maximum age of the news article in days
    search_term = trendKeyword.replace(' ', '+')
    url = f'https://news.google.com/rss/search?q={search_term}&hl=en-US&gl=US&ceid=US:en'
    feed = feedparser.parse(url)
    valid, invalid = [], []
    i = 0
    while i < len(feed.entries) and len(valid) < num_articles:
        entry = feed.entries[i]
        i += 1
        if entry.link not in linkNews:
            pub_date = datetime.datetime.strptime(entry.published, '%a, %d %b %Y %H:%M:%S %Z')
            if datetime.datetime.now() - pub_date > datetime.timedelta(days=maxAge):
                continue

            newNews = News(
                entry.title,
                entry.link,
                entry.published,
            )
            linkNews[entry.link] = newNews
            if newNews.mainText:
                valid.append(newNews)
            else:
                invalid.append(newNews)

    return valid, invalid

In [14]:
def collectArticles(trend, linkNews={}, maxArticles = 30):
    news = set()
    keywords = set(trend['keywords'] + [trend['keyword']])
    numArticlesPerKeyword = 1 + (maxArticles // len(keywords)) # promotes diversity across keywords
    for keyword in keywords:
        time.sleep(2) # wait two seconds to avoid rate limiting
        valid, invalid = getNews(keyword, num_articles=numArticlesPerKeyword, linkNews=linkNews)
        news.update(valid)
        if len(news) > maxArticles:
            break
    trend['news'] = list(news)

In [9]:
def trendTemplate():
    return {'keyword': '', 'volume': -1, 'volume_growth_pct': -1, 'topics': [], 'keywords': []}

# 🔄 Collect Trends Or Load From Pickle
---

In [8]:
getTrendInfo = input('Get Trends? (y/n): ')
if getTrendInfo.lower() == 'y':
    finalTrendList = getTrends()
    showTrends = input('Show Trends? (y/n): ')
    if showTrends.lower() == 'y':
        # outStr = """"""
        for trend in finalTrendList[:min(100, len(finalTrendList))]:
            print(trend)
    saveTrends = input('Save Trends? (y/n): ')
    if saveTrends.lower() == 'y':
        with open('./tmp/finalTrendList.pkl', 'wb') as f:
            pickle.dump(finalTrendList, f)
elif getTrendInfo.lower() == 'n':
    with open('./tmp/finalTrendList.pkl', 'rb') as f:
        finalTrendList = pickle.load(f)
else:
    print('Invalid input, run again')
    exit()

{'keyword': 'social security payments march 26', 'volume': 20000, 'volume_growth_pct': 900, 'topics': [3, 10], 'keywords': ['social security payments march 26']}
{'keyword': 'mexico', 'volume': 10000, 'volume_growth_pct': 600, 'topics': [10], 'keywords': ['mexico', 'yolanda marodi']}
{'keyword': 'vix', 'volume': 5000, 'volume_growth_pct': 400, 'topics': [3], 'keywords': ['vix']}
{'keyword': 'usha vance', 'volume': 5000, 'volume_growth_pct': 100, 'topics': [14, 10], 'keywords': ['usha vance', 'mike waltz']}
{'keyword': 'judge ana reyes', 'volume': 5000, 'volume_growth_pct': 100, 'topics': [10, 14], 'keywords': ['judge ana reyes', 'judge reyes', 'pete hegseth', 'ana reyes', 'u.s. district judge ana reyes']}
{'keyword': 'canada travel warnings', 'volume': 5000, 'volume_growth_pct': 1000, 'topics': [10], 'keywords': ['canada travel warnings', 'travel warnings to usa']}
{'keyword': 'jessica aber us attorney found dead', 'volume': 5000, 'volume_growth_pct': 1000, 'topics': [10], 'keywords': 

In [15]:
trend = trendTemplate()
trend['keyword'] = 'China Taiwan'
finalTrendList.append(trend)

# 🎯 Select Trend and Collect Articles
---

In [16]:
keyword = input('Enter trend keyword: ')
trend = [trend for trend in finalTrendList if trend['keyword'] == keyword][0]
linkNews = {}
collectArticles(trend, linkNews=linkNews, maxArticles = 40)
pickle.dump(finalTrendList, open('./tmp/finalTrendList.pkl', 'wb'))
print('Trends info scraped, trend selected and articles collected. All information saved.')

URL https://news.google.com/rss/articles/CBMieEFVX3lxTE1EcEs0cVp6WXlQTWcxNXRHX1JwSTNQVEctai0tT2FjRTF2MGF4ZjQtY3BEVXloUWVJSkd3dmJVZVUtTl9OcG5Yd1pRYTdrZ1REeWZBN2xBeEFCNmJsd0NTMTNKRWlIRlFXR3dTWWFDTXEweFVwR3pNTA?oc=5
URL https://news.google.com/rss/articles/CBMi1gFBVV95cUxNTXkwNTF6ZVF6eUtNRnlzS3lFdUhkUmRmZ3J1dXE2b245TURzeDFiblc3NVBSc2pwS2c2bi1JaU1yQnNqY1JDT3lUTkJBNEJaZ2UwMjBmSFRVdUp4VFFQNk9odEtKcWlDWFJWU1RKNlJWUUVXOVVVQWMtc0VCVXRTanNsVTVfMDg3S3Z6TnZhWGVNOEdLYTJ5VVo4aHF6aXJSOTU2T095YnUtb1Y4Y2FJN3NEbTRMMk42MEVuTWZlZnlwc0pSSVRpMWljRFdPWDlsUmxpTm1R0gHWAUFVX3lxTE5HSmFXSXkwNUluMXdfLV9fNlVTN0hKdGp1alJyNlBZeDJ1d2RnUmNpZUpsNUU5NGpHWHhqS1EwVi1ySzlkWDVZVTNkbEs2TTI3V3g2ZWlaWEpVUkhkYmlDYjh3LW5SMUV4Z3RSdUI0aW15a3JTNktaVW9wMmhKa2RrcWM2QVM5RDZHZ0NsSTU1MVFrN0psdl9rbTdTRHhCc2lQS0ZFMEE3MHQyeWJueU9aNmZHLU03SVBwWkZrejZ1cFY2NVBkRDhib0VqUEJkczRfVkJ5TkE?oc=5
URL https://news.google.com/rss/articles/CBMif0FVX3lxTE1weUY0a2VKYnlzd0hBbmQ1YW5YV1ZxSHUyZWt2SU1oOU4waWk5VG8xbnlqWlVyeTNwa1RMVFp6MURMazdKc2YwTUpUazRDYmdKRXMt

c:\Users\garre\anaconda3\Lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


URL https://news.google.com/rss/articles/CBMiekFVX3lxTE12WVdza1I1Y1ZqT0FDcWdBVXFMT29uSGZ4dzBNenFnQlZHMWFWWVVSM183RHRTY05PTTBMbVhyQVZLNkZGOTlWZHFhaUM5NS1YSnBLYTlYR194c3lOQU5mZUJ6bFJ3ZWhzeE56YVVvTDlaUzJBV0MwNk93?oc=5
URL https://news.google.com/rss/articles/CBMipAFBVV95cUxOX0dKWHJkUU41Q1hzd3NWbDdUZkNBQmxzeXl6SVFweklIcTQzTTk4cDgzNjFxaUZVYkZuMVZ0RFl0dEN1M3k5cXhlT0FyOWNLTi0xY0U4RTZxR1owcE90ay1FUTMtUVZJNktDWjZTOVc1bXRaUXg5dkpPa0tDcGxDbl9ydm50Wmp1UVA1Ym91MFJUb2xmY0hhUVlVd1d1TXlobXdHag?oc=5
URL https://news.google.com/rss/articles/CBMiZkFVX3lxTFBjTlRpZ1dkdVpjU2RtTUU3Y1lqZGlXRk9PYUlBSmloUy1lN1R3M2FJSFN2UEduSFJRQVhRRC1MbWVqSnJpS2tiY3c3Vm9xS19naFBvRWR2RTQzUGxFbkU4c3ducl9Vdw?oc=5
URL https://news.google.com/rss/articles/CBMiekFVX3lxTE5Nc2REUFJIYzd5RXp4MXFicHRzcmw3TzdZeDR6dTMza1dWZFlxck5FSXJSQW01SmFLclcwR2hEdG1iU211eDdRM2ZQMm81NlNPdHR1M2h0UGNGVmRGcW5qWjBnTkVNTEpfQTR6VmxGN3QxNXhROERVb3BB?oc=5
URL https://news.google.com/rss/articles/CBMiekFVX3lxTE5vQ2MzUG4xX1JGUE40VUxDejBoOGxpc3hybkJGUUJfeDluZXFlVXE

# 🎨 Generate Assets For Selected Trend
---

In [17]:
config = {'news_report': True,
        'script': True,
        'narration': True,
        'voice': False,
        'sora_prompts': True}

In [18]:
print(ELEVENLABS_API_KEY)

sk_f96d619fc51235180796190a025b96954210e5b25af0d67d


In [19]:
generate_assets(trend, openai_api_key=OPENAI_API_KEY, elevenlabs_api_key=ELEVENLABS_API_KEY, voice_id=VOICE_ID, run_dir=RUN_DIR, config=config)

C:\Users\garre\OneDrive\Desktop\Projects\aiMedia\assets\2025-03-23-18-45-56\China_Taiwan topic_dir
C:\Users\garre\OneDrive\Desktop\Projects\aiMedia\assets\2025-03-23-18-45-56\China_Taiwan\sora_prompts sora_prompt_dir
41 validArticles


{'paths': {'script': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-23-18-45-56\\China_Taiwan\\script.txt',
  'audio': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-23-18-45-56\\China_Taiwan\\voiceover.mp3',
  'video': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-23-18-45-56\\China_Taiwan\\video.mp4',
  'sora_prompts': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-23-18-45-56\\China_Taiwan\\sora_prompts',
  'news_report': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-23-18-45-56\\China_Taiwan\\news_report.txt'}}

In [20]:
os.getenv("RUN_DIR")

'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-23-18-45-56'

In [21]:
os.chdir(RUN_DIR)
outName = trend['keyword'].replace(" ", "_") # name of folder and file to zip
!tar -acf {outName}.zip {outName}

In [22]:
!echo python -m wormhole send {outName}.zip | clip && start cmd

# Generating Assets with Grok 🤖
___

In [23]:
import os
from dotenv import load_dotenv
load_dotenv()
from grok import Grok
import pickle

In [24]:
os.chdir(os.getenv("APPDIR"))

In [25]:
grok_api_key, grok_url, grok_model = os.getenv("GROK_API_KEY"), os.getenv("GROK_URL"), 'grok-2-latest'
print(grok_api_key, grok_url)
grok = Grok(grok_api_key, grok_url, grok_model)

xai-fq2OPXVKJrQCMyitw8z4fUiuVE6OO0KBjy1AzT5L56DiZsG88SqNJgvISwXC7psVI6wmT5vkAkFWadXF https://api.x.ai/v1/chat/completions


In [26]:
def generate_script_grok(client: Grok, trend_data: dict, news_report: str) -> str:

    """Generate a video script using Grok based on trend data"""
    
    prompt = f"""Create a short, video script about this trending topic:
    Topic: {trend_data['keyword']}
    Related Keywords: {', '.join(trend_data['keywords'])}
    News Report: {news_report}
    
    The script must be:
    1. Informative and accurate
    2. Focused on the most important and sensational aspects of the story
    3. Conversational and concise.
    4. Written with charged language and emotional appeals.
    5. Quick to the point, no fluff.
    6. Between 60 and 80 words in length.

    The script must NOT:
    1. Contain an outro of any kind, or mention subscribing to the channel
    2. Use acronyms or abbreviations without explanation
    3. Contain any headers, formatting, directions for the video, or other instructions.

    End the script with a uniqueand engaging question.
    """
    return client.generate_text(prompt)

In [27]:
def generate_sora_prompts_grok(client: Grok, script: str, num_prompts: int = 1) -> list[str]:
    """Generate Sora-compatible video prompts based on the script"""

    soraScripts = []
    prompt = f"""Break down this script into a series of four video scene descriptions suitable for AI video generation:
    
    Script: {script}
    
    There should be four scenes.

    For each scene:
    
    1. Describe the visual elements in detail
    2. Include camera movements and transitions
    3. Be concise -- under 25 words.
    4. Use loose, conversational language in your description.
    5. Format the scene description as follows:
        [Scene Title] <= 5 words
        [Scene Description] <= 25 words
        [Camera Movements/Transitions] <= 25 words
        newline

    Do NOT:
    1. Generate a scene for an outro
    2. Use acronyms or abbreviations without explanation
    3. Use the newline character, never the word "newline"
    """
    
    for i in range(num_prompts):
        soraScript = client.generate_text(prompt)
        soraScripts.append(soraScript)
    return soraScripts

In [28]:
def generate_sora_prompts_grokV2(client: Grok, script: str, num_prompts: int = 1) -> list[str]:
    """Generate Sora-compatible video prompts based on the script"""

    soraScripts = []
    prompt = f"""Break down this script into a series of four video scene descriptions suitable for AI video generation:
    
    Script: {script}
    
    There should be five scenes.

    For each scene:
    
    1. Describe the visual elements in detail
    2. Include camera movements and transitions
    3. Be concise -- under 25 words.
    4. Use loose, conversational language in your description.
    5. Scene descriptions should tell a story. Ensure that it can be told in ten seconds or less.
    6. Format the scene description as follows:
        [Scene Title] <= 5 words
        [Scene Description] <= 25 words
        [Camera Movements/Transitions] <= 25 words
        newline

    Do NOT:
    1. Generate a scene for an outro
    2. Use acronyms or abbreviations without explanation
    """
    
    for i in range(num_prompts):
        soraScript = client.generate_text(prompt)
        soraScripts.append(soraScript)
    return soraScripts

In [29]:
def generate_sora_prompts_grokV3(client: Grok, script: str, num_prompts: int = 1, temp = 1.6) -> list[str]:
    """Generate Sora-compatible video prompts based on the script"""

    soraPrompts = []
    prompt = f"""Break down this script into a series of four video scene descriptions suitable for AI video generation:
    
    Script: {script}
    
    There should be four scenes.

    For each scene:
    
    1. Determine the most sensational aspect of the corresponding portion of the script. Capture it with a creative idea described vividly.
    2. Include camera movements and transitions
    3. Be concise -- under 25 words.
    4. Use loose, conversational language in your description.
    5. Format the scene description as follows:
        [Scene Title] <= 5 words
        [Scene Description] <= 25 words
        [Camera Movements/Transitions] <= 25 words
        newline

    Do NOT:
    1. Generate a scene for an outro
    2. Use acronyms or abbreviations without explanation
    3. Use the word "newline". Always use the newline character.
    4. Use people's names. Instead, describe the person in detail.
    """
    
    for i in range(num_prompts):
        soraPrompt = client.generate_text(prompt, temperature=temp)
        print(soraPrompt)
        soraPrompts.append(soraPrompt)
    return soraPrompts

In [30]:
def makeHook(client: Grok, script, audience, temp = 1.6):
    prompt = f"""Generate a short (< 10 words) hook that introduces the following video script. 

    The hook must:
    1. Directly engage the audience, {audience}, with a statement.
    2. Be < 10 words.
    3. Use dramatic language
    4. Capture the most essential themes from the script

    Use a confident tone. DO NOT use a ? anywhere.

    script: {script}
    """
    return client.generate_text(prompt, temperature=temp)

In [31]:
def finalScript(client: Grok, script, hook, temp = 0):
    prompt = f"""Prepend the hook to the script, and make sure it melds well. Do not change the hook.
    
    script: {script}
    hook: {hook}
    """
    return client.generate_text(prompt, temperature=temp)

In [32]:
def generate_grok_image_prompts(client: Grok, script: str, num_prompts: int = 10) -> list[str]:
    """Generate Grok image prompts based on the script"""

    prompt = f"""Generate a series of (~{int}) image descriptions that capture the most sensational themes of the following script.

    For each image:
    1. Be assertive and highly creative in your description. 
    2. Be clear and concise. 
    3. Use familiar elements of popular culture where appropriate e.g. brands, celebrities, landmarks, products, etc.

    Each image prompt should have the following format:
    Generate an image of [description]

    Separate each image prompt with a newline.
    
    Script: {script}
    """
    return client.generate_text(prompt)

In [33]:
def generate_grok_narration(client: Grok, script: str) -> str:
    """Generate a narration using Grok based on the script"""

    prompt = f"""Extract only the words to be spoken from this video script:
    Script: {script}.

    The result should NOT include any mention of a narrator or scene descriptions.
    """
    return grok.generate_text(prompt)

In [34]:
def generate_hashtags(client: Grok, script: str) -> list[str]:
    """Generate hashtags using Grok based on the script"""

    prompt = f"""Generate the best series of hashtags to make this video go viral on instagram reels, tik tok, and youtube shorts.
    Select the best 10 hashtags and brief reasoning for each.

    The result should be in the following format:
    for each hashtag:
        [Hashtag] [Reasoning]

    Include a one-line space separated list of all hashtags at the end.

    Script: {script}"""
    return client.generate_text(prompt)


In [35]:
def generate_grok_opinion_analysis(client: Grok, news_report: str) -> str:
    """Generate an opinion analysis using Grok based on the news report"""

    prompt = f"""Generate an opinion analysis using Grok based on the news report:
    News Report: {news_report}

    The opinion analysis should:
    1. Be highly engaging and bold.
    2. Provide evidence based reasoning for claims made. 
    3. Be less than 500 words in length.
    4. Be written in a professional tone.

    The opinion analysis should NOT:
    1. Cite sources.

    Format:
    [Title Highlighting the Central Claim] <= 10 words
    [Opinion Analysis] <= 500 words
    """

    return client.generate_text(prompt)

In [36]:
grok_api_key, grok_url, grok_model = os.getenv("GROK_API_KEY"), os.getenv("GROK_URL"), 'grok-2-latest'
grok = Grok(grok_api_key, grok_url, grok_model)

In [37]:
# List directories in assets and get most recent
asset_dirs = [d for d in os.listdir('assets') if os.path.isdir(os.path.join('assets', d))]
RUN_DIR = os.path.join('assets', max(asset_dirs))
MAIN = [os.path.join(RUN_DIR, d) for d in os.listdir(RUN_DIR) if os.path.isdir(os.path.join(RUN_DIR, d))][0] # only one directory in the RUN_DIR
NEWS_REPORT = os.path.join(MAIN, 'news_report.txt')
SORAPROMPTS = os.path.join(MAIN, 'soraprompts.txt')
SCRIPT = os.path.join(MAIN, 'script.txt')
keyword = MAIN.split('\\')[-1].replace('_', ' ')

In [38]:
finalTrendList = pickle.load(open('./tmp/finalTrendList.pkl', 'rb'))

In [39]:
trend = [trend for trend in finalTrendList if trend['keyword'] == keyword][0]

In [40]:
audience = "Americans with an interest in the world of business"

In [41]:
with open(NEWS_REPORT) as f:
    newsReport = f.read()
print(newsReport)

### Rising Tensions in Taiwan: President Lai Takes a Hard Stance Against China Amid Military Escalation

**TAIPEI, Taiwan —** In a significant shift in Taiwan's political landscape, President Lai Ching-te has declared China a "foreign hostile force", ramping up national security measures as the threat of Chinese aggression looms large. This declaration, made during a recent high-level national security meeting, marks the first time a Taiwanese leader has openly labeled China in such stark terms, reflecting heightened concerns over espionage and infiltration operations by Beijing within Taiwan.

President Lai's speech outlined a comprehensive 17-point plan aimed at bolstering national security, including proposals to reinstate military courts to address espionage and leaks effectively—a controversial nod to Taiwan's martial law past. He announced a commitment to increase Taiwan's defense budget to over 3% of GDP, a level not seen in more than a decade, amidst U.S. pressure to enhance mi

In [42]:
GROKPATH = MAIN + '/grok'
if not os.path.exists(GROKPATH):
    os.mkdir(GROKPATH)

In [43]:
soraPromptPath = GROKPATH + '/grok-soraprompts.txt'
soraPromptsv2Path = GROKPATH + '/grok-soraprompts-v2.txt'
soraPromptsV3Path = GROKPATH + '/grok-soraprompts-v3.txt'
scriptPath = GROKPATH + '/grok-script.txt'
grokImagePromptsPath = GROKPATH + '/grok-imageprompts.txt'
grokNarrationPath = GROKPATH + '/grok-narration.txt'
grokHashtagsPath = GROKPATH + '/grok-hashtags.txt'
opinionAnalysisPath = GROKPATH + '/grok-opinionanalysis.txt'

In [44]:
narrationWC = 0
while narrationWC < 60 or narrationWC > 80:
    script = generate_script_grok(grok, trend, newsReport)
    narration = generate_grok_narration(grok, script)
    narrationWC = len(narration.split(' '))
    print('narrationWC: ', narrationWC)
    print(narration)
    print("=========")
    
hook = makeHook(grok, narration, audience)
finalNarration = finalScript(grok, script, hook)

with open(scriptPath, 'w') as f:
    f.write(script)
with open(grokNarrationPath, 'w') as f:
    f.write(finalNarration)

narrationWC:  58
Tensions between Taiwan and China are skyrocketing! President Lai Ching-te just called China a "foreign hostile force," sparking outrage from Beijing. Taiwan's beefing up its defenses, with plans to hike military spending and bring back military courts. China's not backing down, ramping up military drills around Taiwan. Will the world stand by as Taiwan fights for its freedom?
narrationWC:  63
Tensions soar as Taiwan's President Lai labels China a "foreign hostile force," sparking a military escalation. Lai's 17-point security plan includes reviving military courts and hiking defense spending to over 3% of GDP. China retaliates with intensified drills, calling Lai a "dangerous separatist." Amidst this, Taiwan's identity shifts, with most now identifying solely as Taiwanese. Can Taiwan's resilience hold against China's looming threat?


In [45]:
images = generate_grok_image_prompts(grok, script)
with open(grokImagePromptsPath, 'w') as f:
    f.write(images)
print(images)

Generate an image of Taiwan's President Lai standing resolutely at a podium, delivering a fiery speech against a backdrop of the Taipei 101 skyscraper illuminated in the colors of the Taiwanese flag, symbolizing national pride and defiance.

Generate an image of a dramatic military parade in Taipei, showcasing Taiwan's advanced weaponry and soldiers in formation, reminiscent of a scene from a Hollywood blockbuster, with the iconic Taipei skyline in the background.

Generate an image of a tense standoff in the Taiwan Strait, with Taiwanese and Chinese naval ships facing off, the tension palpable in the stormy waters, akin to a scene from the movie "Top Gun: Maverick."

Generate an image of a bustling Taiwanese street market, where locals proudly display signs and banners affirming their Taiwanese identity, with popular Taiwanese street foods like stinky tofu and bubble tea prominently featured, symbolizing cultural resilience.

Generate an image of a high-tech Taiwanese military court i

In [46]:
soraPrompts = generate_sora_prompts_grok(grok, script)
with open(soraPromptPath, 'w') as f:
    f.write(soraPrompts[0])
print(soraPrompts[0])

[Tensions Rise]
[President Lai speaks at podium, flags behind, tense atmosphere]
[Camera zooms in on Lai, cuts to crowd reactions]
[Taiwan's Military Plan]
[Documents on table, military officers discussing, charts showing 3% GDP defense spending]
[Camera pans across documents, zooms into charts, cuts to officers]
[China's Retaliation]
[Chinese military drills, ships and planes, aggressive maneuvers, "dangerous separatist" text overlay]
[Camera follows ships, quick cuts between planes and text]
[Taiwan's Identity Shift]
[People on streets, Taiwanese flags, polls showing identity shift]
[Camera moves through crowd, focuses on flags, ends on poll results]


In [47]:
soraPrompts2 = generate_sora_prompts_grokV2(grok, script)
with open(soraPromptsv2Path, 'w') as f:
    f.write(soraPrompts2[0])
print(soraPrompts2[0])

[Tensions Rise in Taiwan]
[President Lai speaks at podium, flags behind, tense atmosphere]
[Camera zooms in on Lai, cuts to crowd reactions]
newline

[Security Plan Unveiled]
[Lai presents 17-point plan on screen, military court scenes]
[Camera pans across plan points, fades to military court]
newline

[China's Military Response]
[Chinese military drills, ships and planes, aggressive maneuvers]
[Camera follows ships, quick cuts to planes in action]
newline

[Taiwanese Identity Shifts]
[People on streets, Taiwanese flags, sense of unity]
[Camera moves through crowd, focuses on flags and faces]
newline

[Resilience Against Threat]
[Taiwanese military on alert, citizens resilient, looming Chinese threat]
[Camera sweeps military, cuts to determined citizens, fades out]
newline


In [48]:
soraPrompts3 = generate_sora_prompts_grokV3(grok, script)
with open(soraPromptsV3Path, 'w') as f:
    f.write(soraPrompts3[0])
print(soraPrompts2[0])

[Tensions Rise]
[The president of Taiwan labels China a "foreign hostile force", standing firm behind a podium.]
[Camera zooms in on the president, then pans to a crowd of supporters nodding in agreement.]

[Security Measures Intensify]
[A group of military officials unveil a 17-point plan, with charts showing defense spending increase to over 3% of GDP.]
[Camera sweeps across the charts, then cuts to soldiers training intensely.]

[China's Retaliation]
[Chinese military jets soar overhead, conducting intensified drills, as a spokesperson calls the Taiwanese leader a "dangerous separatist".]
[Camera follows the jets, then cuts to the spokesperson's stern face on a large screen.]

[Taiwanese Identity Shifts]
[A diverse group of Taiwanese people proudly wave flags and assert their unique identity, symbolizing a shift away from Chinese identity.]
[Camera moves through the crowd, capturing different faces, then pulls back to show the full scope of the gathering.]
[Tensions Rise in Taiwan]


In [49]:
hashtags = generate_hashtags(grok, script)
with open(grokHashtagsPath, 'w') as f:
    f.write(hashtags)
print(hashtags)


1. [#TaiwanChinaTensions] This hashtag directly addresses the central theme of the video, the escalating tensions between Taiwan and China, which is likely to attract viewers interested in geopolitical issues.

2. [#LaiChingte] Including the name of Taiwan's President will draw attention from those specifically interested in his policies and actions, increasing relevance and engagement.

3. [#MilitaryEscalation] This hashtag will attract viewers interested in military developments and conflicts, which is a key element of the video's narrative.

4. [#TaiwanSecurityPlan] By focusing on the specific security measures mentioned in the script, this hashtag will appeal to viewers interested in national security and defense strategies.

5. [#DefenseSpending] This hashtag targets an audience interested in economic aspects of national defense, which is a significant part of the video's content.

6. [#ChinaMilitaryDrills] This will attract viewers who follow military activities and China's strat

In [50]:
opinionAnalysis = generate_grok_opinion_analysis(grok, newsReport)

In [51]:
with open(opinionAnalysisPath, 'w') as f:
    f.write(opinionAnalysis)
print(opinionAnalysis)

**Taiwan's Bold Stance: A Necessary Defense Against Chinese Aggression**

President Lai's resolute declaration of China as a "foreign hostile force" and his comprehensive plan to bolster Taiwan's national security are bold and necessary steps in the face of escalating Chinese aggression. The evidence of China's systematic campaign of infiltration, including increased espionage cases and military drills around Taiwan, underscores the urgency of Lai's actions.

Lai's proposal to reinstate military courts and increase defense spending to over 3% of GDP demonstrates a commitment to protecting Taiwan's sovereignty and democratic values. While controversial, these measures are justified given the severity of the threat posed by China's ambitions for "reunification." The Taiwanese public's growing identification as exclusively Taiwanese further validates the need for a strong national defense strategy.

The international community, particularly the United States, must recognize the gravity of

In [52]:
os.chdir(RUN_DIR)
outName = trend['keyword'].replace(" ", "_") # name of folder and file to zip
!tar -acf {outName}.zip {outName}

# Make Sora Videos 🎥
______

In [53]:
from soraVideoGen import SoraVideoGenerator as SVG
import os
from dotenv import load_dotenv
load_dotenv()

True

In [54]:
os.chdir(os.getenv("APPDIR"))

In [55]:
myuserprofile = r'C:\Users\garre\AppData\Local\Google\Chrome\User Data'

make sure sora is logged in, most recently open in chrome, and set with the appropriate modifiers (display ratio, etc)

In [62]:
svg = SVG(myuserprofile)

In [57]:
ASSETS = os.path.join(os.getcwd(), 'assets')
# Get list of all folders in BASE
folders = [f for f in os.listdir(ASSETS) if os.path.isdir(os.path.join(ASSETS, f))]

# Sort folders by creation time and get most recent
MRF = max(folders, key=lambda x: os.path.getctime(os.path.join(ASSETS, x))) # getctime returns the creation time of the folder 
# MRF = most recent folder

KEYWORD = [f for f in os.listdir(os.path.join(ASSETS, MRF)) if os.path.isdir(os.path.join(ASSETS, MRF, f))][0]
FKP = os.path.join(ASSETS, MRF, KEYWORD) # FULL KEYWORD PATH

In [58]:
prompts = []
promptFiles = [os.path.join(FKP, 'grok', f) for f in os.listdir(os.path.join(FKP, 'grok')) if 'soraprompts' in f]
promptFiles += [os.path.join(FKP, 'sora_prompts', "sora_prompt_0.txt")]

In [59]:
for file in promptFiles:
    with open(file, 'r') as f:
        content = f.read()
        filePrompts = content.split('\n\n')
        for fp in filePrompts:
            prompts.append(fp)

In [60]:
print(*prompts, sep='\n + ====== \n')

[Tensions Rise in Taiwan]
[President Lai speaks at podium, flags behind, tense atmosphere]
[Camera zooms in on Lai, cuts to crowd reactions]
newline
 + ====== 
[Security Plan Unveiled]
[Lai presents 17-point plan on screen, military court scenes]
[Camera pans across plan points, fades to military court]
newline
 + ====== 
[China's Military Response]
[Chinese military drills, ships and planes, aggressive maneuvers]
[Camera follows ships, quick cuts to planes in action]
newline
 + ====== 
[Taiwanese Identity Shifts]
[People on streets, Taiwanese flags, sense of unity]
[Camera moves through crowd, focuses on flags and faces]
newline
 + ====== 
[Resilience Against Threat]
[Taiwanese military on alert, citizens resilient, looming Chinese threat]
[Camera sweeps military, cuts to determined citizens, fades out]
newline
 + ====== 
[Tensions Rise]
[The president of Taiwan labels China a "foreign hostile force", standing firm behind a podium.]
[Camera zooms in on the president, then pans to a cr

In [61]:
svg.submitPrompts(prompts)

[Tensions Rise in Taiwan]
[President Lai speaks at podium, flags behind, tense atmosphere]
[Camera zooms in on Lai, cuts to crowd reactions]
newline
[Security Plan Unveiled]
[Lai presents 17-point plan on screen, military court scenes]
[Camera pans across plan points, fades to military court]
newline
[China's Military Response]
[Chinese military drills, ships and planes, aggressive maneuvers]
[Camera follows ships, quick cuts to planes in action]
newline
[Taiwanese Identity Shifts]
[People on streets, Taiwanese flags, sense of unity]
[Camera moves through crowd, focuses on flags and faces]
newline
[Resilience Against Threat]
[Taiwanese military on alert, citizens resilient, looming Chinese threat]
[Camera sweeps military, cuts to determined citizens, fades out]
newline
[Tensions Rise]
[The president of Taiwan labels China a "foreign hostile force", standing firm behind a podium.]
[Camera zooms in on the president, then pans to a crowd of supporters nodding in agreement.]
[Security Meas

KeyboardInterrupt: 

In [72]:
from grokUtils import generate_sora_prompts_grokV3
from grok import Grok
import utils

In [ ]:
while True:
    sceneDescriptions = generate_sora_prompts_grokV3(grok, get_grok_script(), 1)[0]
    sceneDescriptions = sceneDescriptions.split('\n\n')
    svg.submitPrompts(sceneDescriptions)